In [14]:
%pip install pyspark==3.2.0, requests

Note: you may need to restart the kernel to use updated packages.


In [15]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import *
from requests import get

In [7]:
postgres_jar = "/usr/local/spark/jars/postgresql-42.3.8.jar"

In [8]:
spark = (
    SparkSession
    .builder
    .appName("An Spark application")
    .config("spark.jars", postgres_jar)
    .getOrCreate()
)

25/01/19 23:57:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
jdbc_url = "jdbc:postgresql://postgres-db:5432/downstream"
db_properties = {
    "user": "admin",
    "password": "secret",
    "driver": "org.postgresql.Driver"
}

In [11]:
schema = StructType([
    StructField("Game", StringType(), nullable=True),
    StructField("Year", IntegerType(), nullable=True),
    StructField("Genre", StringType(), nullable=True),
    StructField("Publisher", StringType(), nullable=True),
    StructField("North America", DoubleType(), nullable=True),
    StructField("Europe", DoubleType(), nullable=True),
    StructField("Japan", DoubleType(), nullable=True),
    StructField("Rest of world", DoubleType(), nullable=True),
    StructField("Global", DoubleType(), nullable=True)
])

## Creando la tabla de videojuegos en PSQL

In [12]:
df = spark.read.option("header", True).csv("/home/jovyan/data/PS4_GamesSales.csv")

In [13]:
df.write.jdbc(url=jdbc_url, table="Games", mode="overwrite", properties=db_properties)

## ETL con Spark
Hagamos una ETL usando Spark.
1. Extraemos de MongoDB, nuestro upstream.
2. Realicemos las siguientes transformaciones sobre los datos:
3. Cargamos a PostgreSQL, nuestro downstream.

In [20]:
API_KEY = "4f1e6cf579d44b7db3fe6d6505c54b2b"

In [26]:
def get_news(url: str, search_term: str, date_from: str, api_key: str):
    return get(url.format(search_term, date_from, api_key)).json()

In [34]:
news_url = "https://newsapi.org/v2/everything?q={}&from={}&sortBy=publishedAt&apiKey={}"
get_news(news_url, "elon musk", "2024-12-20", API_KEY)["articles"][0:3]

[{'source': {'id': None, 'name': 'Newsbreak.com'},
  'author': 'Brett Rowland | The Center Square',
  'title': "Report: Federal government can't fully account for its 'unsustainable' spending - NewsBreak",
  'description': "(The Center Square) – A Congressional watchdog says it is again unable to determine if the federal government's financial statements are reliable. The fed",
  'url': 'https://www.newsbreak.com/share/3767247761179-report-federal-government-can-t-fully-account-for-its-unsustainable-spending?_f=app_share&pd=0GOkyYtn&lang=en_US&send_time=1737245806&trans_data=%7B%22platform%22%3A0%2C%22cv%22%3A%2225.3.0.29%22%2C%22languages%22%3A%22en%22%7D&sep=ns_foryou_blend_exp_25q1-v2%2Cns_foryou_model_exp_25q1-v2%2Cns_foryou_rank_exp_25q1-v3%2Cns_foryou_recall_exp_25q1-v5%2Cns_push_exp_rt_bucketv12-v1&s=i2',
  'urlToImage': 'https://img.particlenews.com/img/id/3CHFvx_0yFyNcEn00',
  'publishedAt': '2025-01-19T00:24:36Z',
  'content': "(The Center Square) A Congressional watchdog say